In [4]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
from itertools import chain
import yt.wrapper as yt 
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

In [5]:
def copy_before(table_name):
    if yt.row_count("//home/cloud_analytics/emailing/sender/" + table_name) == 0:
        yt.copy("//home/cloud_analytics/emailing/sender_copy/" + table_name,
                "//home/cloud_analytics/emailing/sender/" + table_name,
                force=True)
        print('saved')
copy_before("Upsell-1-nurture_stream-OLD")
copy_before("Upsell-1-nurture_stream")

# 0. Общая информация

## Drop_list

In [6]:
drop_req = """
SELECT
    *
FROM "//home/cloud_analytics/emailing/sender/drop_list"
FORMAT TabSeparatedWithNames
"""
dropped_emails = lib.execute_query(drop_req)
dropped_emails = set(dropped_emails['email'])

## Cube

In [7]:
req = """
SELECT
    DISTINCT
    billing_account_id,
    lower(user_settings_email) as email,
    puid,
    event_time as console_regstration_date,
    multiIf(first_ba_created_datetime = '0000-00-00 00:00:00', '',
            first_ba_created_datetime) as ba_created,
    
    if (first_first_paid_consumption_datetime == '0000-00-00 00:00:00', '',
        first_first_paid_consumption_datetime) as first_paid_datetime,
    ba_usage_status as usage_status,
    ba_state,
    is_isv,
    mail_feature,
    mail_info,
    mail_promo,
    language,
    segment,
    modulo(toInt64(puid), 100) as group_index,
    toDate(NOW()) - toDate(first_first_paid_consumption_datetime) as
    days_since_first_paid_consumption
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube" as a
ANY LEFT JOIN(
    SELECT
        passport_uid as puid,
        user_settings_language as language
    FROM "//home/cloud_analytics/import/iam/cloud_owners_history"
) as b
ON a.puid == b.puid
WHERE
    (
        (event == 'cloud_created' and  billing_account_id in
         (SELECT DISTINCT billing_account_id 
          FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
          WHERE event == 'cloud_created')
         )
      OR
        (event == 'ba_created' and  billing_account_id not in
         (SELECT DISTINCT billing_account_id 
          FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
          WHERE event == 'cloud_created')
        )
    )
AND puid != ''
AND billing_account_id != ''
AND email != ''
FORMAT TabSeparatedWithNames
"""
main_df = lib.execute_query(req)

# mdb types

In [8]:
mdb_dict = {'mssql':['mssql',
                     'ms sql',
                     'f2emdc91af8p8cpm4tkr',
                     'f2e8cuqa26h8la6c33sq',
                     'f2eovjk4uopcfrsm2e1g'],
    'postgresql': ['postgresql'],
    'mysql': ['mysql'],
    'clickhouse': ['clickhouse'],
    'mongodb': ['mongodb'],
    'data-proc': ['data-proc', 'data proc'],
    'yandex databases': ['ydb', 'yandex database'],
    'redis': ['redis'],
    'kafka': ['kafka'],
    'elastic': ['elastic']
}

In [9]:
matching = [elem for elem in mdb_dict.values()]
matching = str(list(chain.from_iterable(matching)))

In [10]:
def make_if_requests(mdb_dict, event_col = 'event'):
    part_req = ""
    for key in mdb_dict:
        part_req += f"""if(multiMatchAny({event_col}, {str(mdb_dict[key])}), '{key}', '') as "{key}",\n"""
    
    part_req += '"' + '"||"'.join(mdb_dict.keys()) + '" as result,\n'
    return part_req

### 1 type

In [11]:
df_1 = lib.execute_query(f"""
SELECT
    puid,
    groupUniqArray(result) as mdb_create_clusters_links
FROM (
    SELECT
        DISTINCT
        {make_if_requests(mdb_dict)}
        event,
        puid
    FROM "//home/cloud_analytics/import/console_logs/events"
    WHERE  multiMatchAny(event, ['create-cluster', 'create-instance', 'cluster-create', 'create-database'])
    AND multiMatchAny(event, {matching})
    AND event like '%console%'
    AND response >= '200'
    AND response < '300'
    AND toDate(replaceRegexpOne(timestamp, '[.].*', '')) >= addDays(toDate(NOW()), -30)
)
GROUP BY puid
HAVING puid != ''
FORMAT TabSeparatedWithNames
""")

In [12]:
df_1['mdb_create_clusters_links'] = df_1['mdb_create_clusters_links'].astype(str)

### 2 type

In [13]:
df_2 = lib.execute_query(f"""
SELECT
    puid,
    groupUniqArray(result) as mdb_docs_links
FROM (
    SELECT
        DISTINCT
        {make_if_requests(mdb_dict)}
        event,
        puid
    FROM "//home/cloud_analytics/import/console_logs/events"
    WHERE  multiMatchAny(event, ['docs'])
    AND multiMatchAny(event, {matching})
    AND response >= '200'
    AND response < '300'
    AND toDate(replaceRegexpOne(timestamp, '[.].*', '')) >= addDays(toDate(NOW()), -30)
)
GROUP BY puid
HAVING puid != ''
FORMAT TabSeparatedWithNames
""")

In [14]:
df_2['mdb_docs_links'] = df_2['mdb_docs_links'].astype(str)

### 3 type

In [15]:
event_dict = mdb_dict.copy()
event_dict['1C'] = ['1c']
matching = [elem for elem in event_dict.values()]
matching = str(list(chain.from_iterable(matching)))

In [16]:
event_col = 'lower(event_name)'
df_3 = lib.execute_query(f"""
SELECT
    email,
    groupUniqArray(result) as mdb_events
FROM (
SELECT
    DISTINCT
    {make_if_requests(event_dict, event_col=event_col)}
    email
FROM "//home/cloud_analytics/export/marketo/ya_attend_event"
WHERE multiMatchAny({event_col}, {matching})
AND toDate(event_date) >= toDate('2020-01-01')
)
GROUP BY email
HAVING email != ''
FORMAT TabSeparatedWithNames
""")
df_3['mdb_events'] = df_3['mdb_events'].astype(str)

### 4 type

In [17]:
req = """
SELECT
    billing_account_id,
    arraySort(arrayDistinct(groupArray(db))) as mdb_data_bases_on_vm
FROM "//home/cloud_analytics/import/network-logs/db-on-vm/data"
WHERE
    billing_account_id IS NOT NULL
    AND billing_account_id != ''
GROUP BY
    billing_account_id
FORMAT TabSeparatedWithNames
"""
df_4 = lib.execute_query(req)
df_4['mdb_data_bases_on_vm'] = df_4['mdb_data_bases_on_vm'].astype(str)

### 5 type

In [18]:
df_5 = lib.execute_query("""
SELECT
    groupUniqArray(dimension_name) as mdb_dimensions,
    billing_account_id
FROM "//home/cloud_analytics/kulaga/leads_cube"
WHERE dimension_name in ('Веб-сайт', 'Веб-приложение', '1С хостинг')
GROUP BY billing_account_id
HAVING hasAny(mdb_dimensions, ['Веб-сайт', 'Веб-приложение', '1С хостинг'])
FORMAT TabSeparatedWithNames""".encode('utf-8'))
df_5['mdb_dimensions'] = df_5['mdb_dimensions'].astype(str)

### 6 type

In [19]:
solutions_dict = {'Веб':['web'],
                  '1С': ['1C', 'internet-store/bitrix-shop', '1c'],
                  'datalens': ['datalens'],
}

In [20]:
df_6 = lib.execute_query(f"""
SELECT
    puid,
    groupUniqArray(result) as mdb_solutions_links
FROM (
    SELECT
        DISTINCT
        {make_if_requests(solutions_dict)}
        event,
        puid
    FROM "//home/cloud_analytics/import/console_logs/events"
    WHERE  multiMatchAny(event, ['infrastructure-management/1c-mssql-windows',
                                 'infrastructure-management/1c-postgresql-linux',
                                 'internet-store/bitrix-shop',
                                 'web/bitrix-website',
                                 'web/joomla-postgresql',
                                 'web/wordpress-mysql',
                                 'solutions/dat alens'])
    AND event like '%solutions%'
    AND response >= '200'
    AND response < '300'
    AND toDate(replaceRegexpOne(timestamp, '[.].*', '')) >= addDays(toDate(NOW()), -30)
)
GROUP BY puid
HAVING puid != ''
FORMAT TabSeparatedWithNames
""".encode('utf-8'))
df_6['mdb_solutions_links'] = df_6['mdb_solutions_links'].astype(str)

### Потребление

In [21]:
sku_dict = {'mssql':['\.ms.sql\.'],
    'postgresql': ['\.pg\.'],
    'mysql': ['\.mysql\.'],
    'clickhouse': ['\.clickhouse\.'],
    'mongodb': ['\.mongodb\.'],
    'data-proc': ['\.dataproc\.'],
    'redis': ['\.redis\.'],
    'kafka': ['\.kafka\.']
}
matching = [elem for elem in sku_dict.values()]
matching = str(list(chain.from_iterable(matching)))
matching = matching.replace("\\\\", '\\')

In [22]:
def cons_req(sku_dict, matching):
    part_req = ""
    for key in sku_dict:
        curr_str = str(sku_dict[key]).replace("\\\\", '\\')
        part_req += f"""MAX(if(multiMatchAny(sku_name, {curr_str}), '{key}', '')) as "{key}",\n"""
    
    part_req += '''arrayFilter(x -> x != '', ["''' + \
                '","'.join(sku_dict.keys()) + '"]) as result,\n'
    part_req += f"""SUM(IF(multiMatchAny(sku_name, {matching}), trial_consumption, 0)) as trial_mdb_cons,\n"""
    part_req += f"""SUM(IF(multiMatchAny(sku_name, {matching}), real_consumption, 0)) as paid_mdb_cons,\n"""
    return part_req

In [23]:
cons_df = lib.execute_query(f"""
SELECT
    billing_account_id,
    result as used_mdb_last_month,
    trial_mdb_cons,
    paid_mdb_cons,
    if(paid_mdb_cons < 100, 1, 0) as zero_mdb_paid_consumption,
    if(paid_mdb_cons > 2000, 1, 0) as mdb_progress_programm,
    is_churned
FROM (
SELECT
    {cons_req(sku_dict, matching)}
    billing_account_id,
    if (sum(real_consumption + trial_consumption) < 1, 1, 0) as is_churned 
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
WHERE toDate(event_time) >= addDays(toDate(NOW()), -30)
GROUP BY billing_account_id
)
FORMAT TabSeparatedWithNames  
""")
cons_df['used_mdb_last_month'] = cons_df['used_mdb_last_month'].astype(str)

## Mergering

In [24]:
merged = [(cons_df, "billing_account_id"), 
          (df_1, 'puid'), (df_2, 'puid'), (df_3, 'email'), 
          (df_4, 'billing_account_id'), (df_5, 'billing_account_id'), (df_6, 'puid')]

In [25]:
mdb_table = main_df.copy()
for table, key in merged:
    mdb_table = pd.merge(mdb_table, table, on=key, how='left')

# Kuber

### 1 type

In [26]:
kuber_1 = lib.execute_query("""
SELECT
    billing_account_id,
    MAX(if(service_name == 'cr', 1, 0)) as is_container_reg_used,
    SUM(if(service_name == 'mk8s', real_consumption, 0)) as kuber_cons,
    IF(kuber_cons < 2000, 1, 0) as for_kuber_upsell,
    IF(kuber_cons < 100, 1, 0) as zero_kuber_paid_consumption
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
WHERE toDate(event_time) >= addDays(toDate(NOW()), -30)
GROUP BY billing_account_id
FORMAT TabSeparatedWithNames  
""")

### 2 type

In [27]:
kuber_2 = lib.execute_query(f"""
SELECT
    DISTINCT
    1 as kuber_link,
    puid
FROM "//home/cloud_analytics/import/console_logs/events"
WHERE multiMatchAny(event, ['kubernetes'])
AND response >= '200'
AND response < '300'
AND puid != ''
AND toDate(replaceRegexpOne(timestamp, '[.].*', '')) >= addDays(toDate(NOW()), -30)
FORMAT TabSeparatedWithNames
""")

### 3 type

In [28]:
kuber_3 = lib.execute_query(f"""
SELECT
    DISTINCT
    1 as kuber_event,
    email
FROM "//home/cloud_analytics/export/marketo/ya_attend_event"
WHERE (multiMatchAny(lower(event_name), ['kuber']) 
or multiMatchAny(event_link, ['events/81', 'events/80', 'events/84']))
AND email != ''
AND toDate(event_date) >= toDate('2020-01-01')
FORMAT TabSeparatedWithNames
""")

### 4 type

In [29]:
kuber_4 = lib.execute_query(f"""
SELECT
    ba_id as billing_account_id,
    COUNT(DISTINCT node_az) as is_1_az
FROM "//home/cloud_analytics/compute_logs/vm_cube/vm_cube"
GROUP BY billing_account_id
HAVING is_1_az == 1
FORMAT TabSeparatedWithNames
""")

## Mergering  kuber

In [30]:
merged = [(kuber_1, "billing_account_id"), 
          (kuber_2, 'puid'), (kuber_3, 'email'), (kuber_4, 'billing_account_id')]

In [31]:
all_table = mdb_table.copy()
for table, key in merged:
    all_table = pd.merge(all_table, table, on=key, how='left')


In [32]:
main_table = all_table.copy()

In [33]:
assert main_table.shape[0] == mdb_table.shape[0]

In [34]:
main_table.replace('[]', '', inplace=True)
for col in main_table.columns:
    if main_table[col].dtype == 'object':
        main_table[col] = main_table[col].fillna('')
    else:
        if 'zero_' in col or 'for_kuber_upsell' in col or 'churned' in col:
            main_table[col] = main_table[col].fillna(1)
        else:
            main_table[col] = main_table[col].fillna(0)

In [35]:
assert main_table.isna().sum().sum() == 0

In [36]:
df = main_table.copy()

In [37]:
def find_group(row):
    if row['is_churned'] == 1:
        return "Churn"
    
    if (row['kuber_event'] == 1 or \
       row['is_container_reg_used'] == 1 or row['kuber_link'] == 1):
        if row['zero_kuber_paid_consumption'] == 1:
            return 'Upsell-Kubernetes-first'
        if row['for_kuber_upsell'] == 1:
            return 'Upsell-Kubernetes-start'
        
    mdb_type = ''
    if row['mdb_progress_programm'] == 1:
        mdb_type = "Upsell-MDB-progress"
    if row['zero_mdb_paid_consumption'] == 1:
        mdb_type = 'Upsell-MDB-first'
    if row['zero_mdb_paid_consumption'] == 0 and row['mdb_progress_programm'] == 0:
        mdb_type="Upsell-MDB-start"

    if row['mdb_create_clusters_links'] != '' or row['mdb_data_bases_on_vm'] != '':
        return mdb_type
        
    experiment = []
    if row['mdb_solutions_links'] != '' or row['mdb_dimensions'] != '' or\
       row['mdb_events'] != '' or row['mdb_docs_links'] != '':
        experiment.append(mdb_type)

    if len(experiment) == 1:
        return experiment[0]
    elif len(experiment) == 2:
        ind = int(np.random.rand() > 0.5)
        return experiment[ind]
    else:
        return 'Upsell-Other'

In [38]:
df['Upsell_type'] = df.apply(lambda row: find_group(row), axis=1)

In [39]:
main_df = df.copy()

In [40]:
main_df.shape

(125809, 36)

# 1. Обновление

In [41]:
def update_current_table(table_name, path, tables_to_update):
    full_path = path + "/" + table_name
    req = f"""
    SELECT
        *
    FROM "{full_path}"
    FORMAT TabSeparatedWithNames
    """
    old_df = lib.execute_query(req)
    if 'sended_mails' in old_df.columns:
        old_df['sended_mails'] = old_df['sended_mails'].apply(
            lambda x: x.replace("\\", ""))
    
    if 'upsell_3_users' in old_df.columns:
        old_df.drop(columns = ['upsell_3_users'], inplace=True)
    old_df['is_dropped'] = (old_df['email'].isin(dropped_emails)).astype(int)
    for table, key in tables_to_update:
        old_df = pd.merge(old_df,
                          table, on=key, suffixes=('', '_new'), how='left')
        
    new_cols = []
    if 'Upsell_type_new' in old_df.columns:
        old_df.drop(columns=['Upsell_type_new'], inplace=True)
    for column in old_df.columns:
        if "_new" == column[-4:]:
            new_cols.append(column)
            old_column = column[:-4]
            old_df[old_column] = old_df[column].copy()
    old_df.drop(columns=new_cols, inplace=True)
    old_df.dropna(inplace=True)
    old_df = old_df.groupby('email').tail(1)
    lib.save_table(table_name, path, old_df)

In [42]:
# marketo_previous_req = """
# SELECT
#     DISTINCT
#     billing_account_id,
#     any(email) as email,
#     groupUniqArray(if(mailing_name == 'inapplicable', null, mailing_name)) as sended_mails,
#     if (length(sended_mails)  > 0, 'test', 'control') as Group,
#     max(if (lower(program_name) like '%-upsell-1%', toDate(event_time), null)) as experiment_date
# FROM "//home/cloud_analytics/cubes/emailing_events/emailing_events"
# WHERE lower(program_name) like '%-upsell-1%'
# OR lower(mailing_name) like '%-upsell-1%'
# GROUP BY billing_account_id
# HAVING isNotNull(experiment_date)
# FORMAT TabSeparatedWithNames
# """
# marketo_prev_df = lib.execute_query(marketo_previous_req)
# marketo_prev_df['sended_mails'] = marketo_prev_df['sended_mails'].astype(str)
# lib.save_table("Upsell-1-nurture_stream-OLD", "//home/cloud_analytics/emailing/sender", 
#                marketo_prev_df)

In [43]:
tables_to_update = [(main_df, "billing_account_id")]

In [44]:
assert main_df.isna().sum().sum() == 0

In [45]:
update_current_table("Upsell-1-nurture_stream-OLD",
                     "//home/cloud_analytics/emailing/sender", 
                     tables_to_update)

In [46]:
update_current_table("Upsell-1-nurture_stream",
                     "//home/cloud_analytics/emailing/sender", 
                     tables_to_update)

# 2. Добавление

## old Upsell Marketo

In [47]:
marketo_previous_req = """
SELECT
    DISTINCT
    billing_account_id,
    email
FROM "//home/cloud_analytics/cubes/emailing_events/emailing_events"
WHERE lower(program_name) like '%-upsell-1%'
FORMAT TabSeparatedWithNames
"""
marketo_prev_df = lib.execute_query(marketo_previous_req)

In [48]:
marketo_prev_df.shape

(6973, 2)

## old in program


In [49]:
req = """
SELECT
    DISTINCT
        email,
        billing_account_id
FROM "//home/cloud_analytics/emailing/sender/Upsell-1-nurture_stream"
FORMAT TabSeparatedWithNames
"""
old_df = lib.execute_query(req)

In [50]:
req = """
SELECT
    *
FROM "//home/cloud_analytics/emailing/sender/Upsell-1-nurture_stream"
FORMAT TabSeparatedWithNames
"""
old_df = lib.execute_query(req)

In [51]:
old_df.shape

(7447, 39)

In [54]:
new_table = main_df.copy()
#print("Все пользователи:", new_table.shape[0])
new_table = new_table[
    (~new_table['email'].isin(marketo_prev_df['email'])) &
    (~new_table['billing_account_id'].isin(marketo_prev_df['billing_account_id']))
]
#print("Все пользователи после удаления тех, кто уже был в старом стриме upsell:", new_table.shape[0])
new_table = new_table[~new_table['email'].isin(dropped_emails)]
#print("Отказались от рассылок по маркето информации:", new_table.shape[0])
new_table = new_table[
    (~new_table['email'].isin(old_df['email'])) &
    (~new_table['billing_account_id'].isin(old_df['billing_account_id']))
]
new_table = new_table[(new_table['is_isv'] == 0) &
                      (new_table['first_paid_datetime'] != '')]
#print("Есть платное потребление, не isv:", new_table.shape[0])
new_table = new_table[(new_table['mail_feature'] == 1) |
                      (new_table['mail_info'] == 1) |
                      (new_table['mail_promo'] == 1)]
#print("Есть согласие на рассылку:", new_table.shape[0])
new_table = new_table[new_table['days_since_first_paid_consumption'] >= 10]
#print("Прошло хотя бы 10 дней с первого платного потребления:", new_table.shape[0])
new_table['Group'] = new_table['puid'].astype(int).apply(
    lambda x: 'control' if x % 100 >= 35 and x % 100 <= 44 else 'test')

In [50]:
new_table['experiment_date'] = lib.get_current_date_as_str()
new_table['is_dropped'] = (new_table['email'].isin(dropped_emails)).astype(int)
new_table = new_table.groupby('email').tail(1)

In [51]:
new_table.shape

(0, 39)

In [52]:
lib.save_table("Upsell-1-nurture_stream", "//home/cloud_analytics/emailing/sender", 
               new_table, append=True)

In [64]:
def copy_after(table_name):
    if yt.row_count("//home/cloud_analytics/emailing/sender/" + table_name) != 0:
        yt.copy("//home/cloud_analytics/emailing/sender/" + table_name,
                "//home/cloud_analytics/emailing/sender_copy/" + table_name, force=True)
copy_after("Upsell-1-nurture_stream-OLD")
copy_after("Upsell-1-nurture_stream")

# 6 стримов

In [65]:
common_part = "//home/cloud_analytics/emailing/sender_copy/" 
old_name = "Upsell-1-nurture_stream-OLD"
new_name = "Upsell-1-nurture_stream"


def split_in_tables(name):
    path = common_part + name
    final_common_df = lib.execute_query(f"""
    SELECT
        billing_account_id,
        email,
        experiment_date,
        Group,
        Upsell_type
    FROM "{path}"
    FORMAT TabSeparatedWithNames
    """)
    for exp_name, table in final_common_df.groupby('Upsell_type'):
        lib.save_table(name + "_" + exp_name, "//home/cloud_analytics/emailing/sender",  table)

In [66]:
split_in_tables(old_name)
split_in_tables(new_name)